## Упражнение 01: Подзапрос

In [23]:
import pandas as pd
import sqlite3

## создаем подключение к базе данных с помощью библиотеки sqlite3

In [24]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

## получить схему проверки таблицы

In [25]:
pd.read_sql('pragma table_info(checker);', conn)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,status,TEXT,0,None,0
2,2,success,INTEGER,0,None,0
3,3,timestamp,TIMESTAMP,0,None,0
4,4,numTrials,INTEGER,0,None,0
5,5,labname,TEXT,0,None,0
6,6,uid,TEXT,0,None,0


## получить только первые 10 строк таблицы checker, чтобы проверить, как выглядит таблица

In [26]:
pd.read_sql('select * from checker limit 10;', conn)

,index,status,success,timestamp,numTrials,labname,uid
0,0,checking,0,2020-04-16 21:12:50.740474,5,None,admin_1
1,1,ready,0,2020-04-16 21:12:54.708365,5,code_rvw,admin_1
2,2,checking,0,2020-04-16 21:46:47.769088,7,None,admin_1
3,3,ready,0,2020-04-16 21:46:48.121217,7,lab02,admin_1
4,4,checking,0,2020-04-16 21:53:01.862637,6,code_rvw,admin_1
5,5,ready,0,2020-04-16 21:53:05.373389,6,code_rvw,admin_1
6,6,checking,0,2020-04-17 05:18:51.965864,1,None,None
7,7,ready,0,2020-04-17 05:19:02.744528,1,project1,user_4
8,8,checking,0,2020-04-17 05:22:35.249331,2,project1,user_4
9,9,ready,1,2020-04-17 05:22:45.549397,2,project1,user_4


## подсчитайте, сколько строк удовлетворяют следующим условиям, используя только один запрос с любым количеством подзапросов:
подсчитать строки из таблицы просмотров страниц, но только с пользователями из таблицы проверки с:
- status = 'ready', мы не хотим анализировать журналы, находящиеся на проверке статуса
- numTrials = 1, мы хотим проанализировать только первые коммиты, потому что только они могут сказать нам, когда студент начал работать над лабораторной работой
- labnames должны быть из списка: 'laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1'. Только они были активны во время эксперимента

сохранить в фрейме данных checkers со столбцом cnt

In [27]:
query = """
        select count(*) as cnt
        from pageviews
        where uid in (select uid from checker
         where status = 'ready' and numTrials = 1
         and labname in ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')); 
        """

chekers = pd.read_sql(query, conn)
chekers

,cnt
0,985


## закрыть соединение

In [28]:
conn.close()